In [5]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import folium
import ee
%matplotlib inline

In [6]:
# initialize GEE using project's service account and JSON key
service_account = "climateeye@ee-test-361116.iam.gserviceaccount.com"
json_key = "/Users/emeliamavis/ee-test-361116-e5216028be0e.json"
ee.Initialize(
    ee.ServiceAccountCredentials(service_account, json_key),
        opt_url="https://earthengine-highvolume.googleapis.com",
    )  

In [7]:
# define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)

# add ee drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [9]:
# define a region of interest (AOI) as long-lat corners of the Frankfurt Airport as example.
# geojson.io webset provided below code snippet of the corresponding GeoJSON object description
# note first and last corners are identical (closing of the polygon)
geoJSON = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [8.473892211914062, 49.98081240937428],
                        [8.658599853515625, 49.98081240937428],
                        [8.658599853515625, 50.06066538593667],
                        [8.473892211914062, 50.06066538593667],
                        [8.473892211914062, 49.98081240937428],
                    ]
                ],
            },
        }
    ],
}


In [10]:
# create ee.Geometry() object
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [13]:
# filter sentinel-1 archive to get an image over the aoi acquired sometime in Aug 2020 (we do not specify orbit number nor the ascending or descending node)
# if we don't specify instrument mode or resolution, default is IW (interferometric wide swath) mode and 10m x 10m pixels

# decibel version
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')).first().clip(aoi))

# float version
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterBounds(aoi).filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')).first().clip(aoi))

In [15]:
# confirm we have an image
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [16]:
# display the image
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [17]:
# use folium to project image onto a map
location = aoi.centroid().coordinates().getInfo()[::-1]

# make an RGB color composite image (VV, VH, VV/VH)
rgb = ee.Image.rgb(ffa_db.select('VV'), ffa_db.select('VH'), ffa_db.select('VV').divide(ffa_db.select('VH')))

# create map object
m = folium.Map(location=location, zoom_start=12)

# add the sentinel1 rgb composite to the map object
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# add a later control panel to the map
m.add_child(folium.LayerControl())

# display the map
display(m)